In [7]:
import allel
import numpy as np
import pandas as pd
from itertools import combinations

In [2]:
encoded = pd.read_csv("encoded.csv", header=0, names=['rsID', 'sequence'])
encoded.head()

,rsID,sequence
0,rs577013928,ttttttttttttttttttttttttttttttttttttttttttttt...
1,rs565082899,ttttttttttttttttttttttttttttttttttttttttttttt...
2,rs540382744,ttttttttttttttttttttttttttttttttttttttttttttt...
3,rs573244332,ttttttttttttttttttttttttttttttttttttttttttttt...
4,rs539162497,ttttttttttttttttttttttttttttttttttttttttttttt...


In [3]:
def makeArray(strings):
    
    # initialize sample array with empty list
    bebG, cheG, esnG, gbrG, pelG = [], [], [], [], []
    bebH, cheH, esnH, gbrH, pelH = [], [], [], [], []
    
    # swap the keys and values in the dictionary below
    nested_dictionary = {'GBR':{'e':'a', 'f':'b', 'g':'c', 'h': 'd'},
                         'PEL':{'i':'a', 'j':'b', 'k':'c', 'l': 'd'},
                         'ESN':{'m':'a', 'n':'b', 'o':'c', 'p': 'd'},
                         'BEB':{'q':'a', 'r':'b', 's':'c', 't': 'd'},
                         'CHE':{'u':'a', 'v':'b', 'w':'c', 'x': 'd'}}

    ginfo = {'a':'1|0', 'b':'1|1', 'c':'0|1', 'd':'0|0'}

    dic = {'e':'GBR', 'f':'GBR', 'g':'GBR', 'h':'GBR',
           'i':'PEL', 'j':'PEL', 'k':'PEL', 'l':'PEL',
           'm':'ESN', 'n':'ESN', 'o':'ESN', 'p':'ESN',
           'q':'BEB', 'r':'BEB', 's':'BEB', 't':'BEB',
           'u':'CHE', 'v':'CHE', 'w':'CHE', 'x':'CHE'}

    for string in strings:
        string = string.strip()
        genotypes = {}
        N = len(string)
         # initialize haplotype dictionary with 462 zeros for each sample
        haplotypes = {'BEB':[0]*N, 'CHE':[0]*N, 'ESN':[0]*N, 'GBR':[0]*N, 'PEL':[0]*N}

        for i, c in enumerate(string):
            # take sample as key and 0|1 etc as value
            genotypes.setdefault(dic[c], []).append(ginfo[nested_dictionary[dic[c]][c]])
            # set 1 in which position sample present
            haplotypes[dic[c]][i] = 1

        # sort dictionary alphabetically
        genotypes = dict(sorted(genotypes.items(), key = lambda x:x[0].lower()))
        # split each 0|1 etc by | and make a list,,,thus apply on each sample
        genotype_array = [[list(map(int, v.split('|'))) for v in val] for val in genotypes.values()]
        # create haplotype array
        haplotype_array = [val for val in haplotypes.values()]

        # extracting genotype array samples
        bebG.append(genotype_array[0])
        cheG.append(genotype_array[1])
        esnG.append(genotype_array[2])
        gbrG.append(genotype_array[3])
        pelG.append(genotype_array[4])
        
        # extracting haplotype array samples
        bebH.append(haplotype_array[0])
        cheH.append(haplotype_array[1])
        esnH.append(haplotype_array[2])
        gbrH.append(haplotype_array[3])
        pelH.append(haplotype_array[4])
        
    return (bebG, cheG, esnG, gbrG, pelG)

In [4]:
# passing first 3 sequences into makeArray function
g = makeArray(encoded['sequence'].iloc[100:200]) # we can pass a list inside the function as [snp1, snp2, snp3, ...]

# extract genotype array into samples
bebG, cheG, esnG, gbrG, pelG = g

# extract haplotype array into samples
# bebH, cheH, esnH, gbrH, pelH = h

## Haplotype Diversity

These function calculate an estimate of Haplotype Diversity

In [5]:
def haplotype_diversity(strings):
    # passing sequences into makeArray function
    g = makeArray(strings)
    # extract genotype array into samples
    bebG, cheG, esnG, gbrG, pelG = g
    
    haplotype_diversity = {}
    
    # for each population calculate haplotype diversity
    for pair,val in zip( combinations(['bebG','cheG','esnG','gbrG','pelG'],1), combinations([bebG,cheG,esnG,gbrG,pelG],1)):
        
        # biuld genotype array of list of genotypes
        g = allel.GenotypeArray(val[0])
        
        # convert genotype array into haplotype array
        haplo = g.to_haplotypes()
        
        #calculate haplotype_diversity inside the region selected
        div = allel.haplotype_diversity(haplo)
    
        # update dictionary for returning
        haplotype_diversity.update({str(pair).strip("(''),") : div})
    
    return haplotype_diversity

#### 4 SNP region 
all of the are most likely only 0|0

In [16]:
haplotype_diversity(encoded['sequence'].iloc[0:4])



{'bebG': 0.0, 'cheG': 0.0, 'esnG': 0.0, 'gbrG': 0.0, 'pelG': 0.0}

#### 6 SNP region

In [17]:
haplotype_diversity(encoded['sequence'].iloc[4:10])



{'bebG': 0.0,
 'cheG': 0.0,
 'esnG': 0.010101010101010152,
 'gbrG': 0.0,
 'pelG': 0.0}

#### 2 SNP region with lots of 1|1, 1|0 etc...

In [24]:
haplotype_diversity(encoded['sequence'].iloc[11:13])



{'bebG': 0.39412484700122413,
 'cheG': 0.49031494198437137,
 'esnG': 0.4163462031482336,
 'gbrG': 0.19786291057009295,
 'pelG': 0.4722589627567003}

#### 100 SNP region

values differ more

In [8]:
haplotype_diversity(encoded['sequence'].iloc[100:200])



{'bebG': 0.6223310213518293,
 'cheG': 0.48496329623490403,
 'esnG': 0.9390350202532943,
 'gbrG': 0.8314613563232347,
 'pelG': 0.7495997215454231}

#### 1000 SNP region

Values nearly 1

In [11]:
haplotype_diversity(encoded['sequence'].iloc[1000:2000])



{'bebG': 0.9812321501427989,
 'cheG': 0.9846080985081695,
 'esnG': 0.9865148951443368,
 'gbrG': 0.9803290632019914,
 'pelG': 0.9429864253393666}

#### 19k SNP region
Values = 1



In [12]:
haplotype_diversity(encoded['sequence'].iloc[1000:20000])



{'bebG': 1.0, 'cheG': 1.0, 'esnG': 1.0, 'gbrG': 1.0, 'pelG': 1.0}

## Moving Haplotype Diversity

In [14]:
def moving_haplotype_diversity(strings):
    # passing sequences into makeArray function
    g = makeArray(strings)
    # extract genotype array into samples
    bebG, cheG, esnG, gbrG, pelG = g
    
    haplotype_diversity = {}
    
    # for each population calculate haplotype diversity
    for pair,val in zip( combinations(['bebG','cheG','esnG','gbrG','pelG'],1), combinations([bebG,cheG,esnG,gbrG,pelG],1)):
        
        # biuld genotype array of list of genotypes
        g = allel.GenotypeArray(val[0])
        
        # convert genotype array into haplotype array
        haplo = g.to_haplotypes()
        
        #calculate haplotype_diversity inside the region selected
        div = allel.moving_haplotype_diversity(haplo, 500)
        
        # update dictionary for returning
        haplotype_diversity.update({str(pair).strip("(''),") : div})
    
    return haplotype_diversity

#### 100 SNP region

with 25 snps window

In [10]:


moving_haplotype_diversity(encoded['sequence'].iloc[100:200])



{'bebG': array([0.3498572 , 0.46838025, 0.23453012, 0.27410581]),
 'cheG': array([0.10286526, 0.38493962, 0.34719394, 0.33393322]),
 'esnG': array([0.64682357, 0.87709583, 0.60103574, 0.1976619 ]),
 'gbrG': array([0.43731407, 0.65229798, 0.23022282, 0.55977172]),
 'pelG': array([0.56950922, 0.53233554, 0.11200835, 0.22784546])}

#### 1000 SNP region
with 25 snps window

In [13]:
moving_haplotype_diversity(encoded['sequence'].iloc[1000:2000])



{'bebG': array([0.3119135 , 0.        , 0.62015504, 0.69386645, 0.01162791,
        0.54141167, 0.04603563, 0.20916633, 0.        , 0.68203454,
        0.1121991 , 0.73534612, 0.54222766, 0.49578403, 0.64776282,
        0.38283694, 0.01162791, 0.07853937, 0.52638379, 0.40248878,
        0.01162791, 0.45525636, 0.01162791, 0.60397117, 0.70433837,
        0.5248878 , 0.39419285, 0.60165919, 0.28967768, 0.43825649,
        0.55637155, 0.02318781, 0.16109071, 0.02311982, 0.76968584,
        0.02318781, 0.56922345, 0.65347477, 0.60988712, 0.09037128]),
 'cheG': array([0.38034573, 0.        , 0.66483543, 0.66819796, 0.00970874,
        0.52450864, 0.18816008, 0.35917594, 0.16107033, 0.64106086,
        0.16987923, 0.75808667, 0.47951693, 0.33341227, 0.59829505,
        0.2500592 , 0.00970874, 0.15292446, 0.51991475, 0.25834715,
        0.01937012, 0.2500592 , 0.04792801, 0.26654037, 0.611982  ,
        0.33175468, 0.25271134, 0.60520957, 0.3318494 , 0.36130713,
        0.50144447, 0.00970874

### 19000 SNPs region (window size changed to 500 for this)
cause with 25 it would be a mess.



As you can see the values in windows of 500 are getting very close to 1

In [15]:
moving_haplotype_diversity(encoded['sequence'].iloc[1000:20000])



{'bebG': array([0.92846457, 0.95920033, 0.95321637, 0.98306814, 0.96627227,
        0.94498844, 0.97171223, 0.94573643, 0.97327621, 0.999592  ,
        0.99966   , 0.97572419, 0.999932  , 0.999864  , 0.999524  ,
        0.99870801, 0.999728  , 0.999592  , 0.96858425, 0.97599619,
        0.98551612, 0.96858425, 0.94104447, 0.98966408, 0.999456  ,
        0.99728002, 0.93172855, 0.92166463, 0.98844009, 0.96362029,
        0.98918809, 0.97096423, 0.90527676, 0.9498164 , 0.96851625,
        0.96328029, 0.94063647, 0.9128927 ]),
 'cheG': array([0.96187544, 0.94283685, 0.9252664 , 0.98332939, 0.96907412,
        0.91835188, 0.98304523, 0.92564528, 0.95216671, 0.99829505,
        0.9997632 , 0.97537296, 0.99981056, 0.9995264 , 0.99763202,
        0.99616386, 0.99943168, 0.99971584, 0.96220696, 0.96220696,
        0.96973715, 0.9775515 , 0.95680796, 0.99180677, 0.9975373 ,
        0.99853185, 0.92569264, 0.91565238, 0.98342411, 0.95316126,
        0.96358039, 0.96078617, 0.95941274, 0.94847265

### Conclusion

The bigger the region the bigger the values cause the propability of picking two different alleles gets higher.
The results should be correct the only issue I have rn is when I scroll through the VCF file I actually see so many 0|0 that it makes little sense to me, so I would say the propability gets lower for this. On the other hand it is just manualy scrolling and selecting the first 4 rsIDs makes the values very low which makes it valid in my opinion cause these are pretty only 0|0 for the most pops. In total I wouldnt expect these high values but when you look into google and papers there are often these high values presented. 